## Import Relevant Packages

In [ ]:
# Base Packages
import numpy as np
np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)
import pandas as pd
pd.options.display.max_rows = 200

# File IO and System Packages
import dill

# File Download from Website 
import requests

# Pandas Formatting and Styling:
pd.options.display.max_columns = 500
pd.set_option('display.float_format',lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings('ignore')

# Geo Packages
import fiona
import geopandas as gpd
import geopy as gpy
from geopy.geocoders import Nominatim
import shapely
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union
import shapefile
import pysal as ps

In [ ]:
# Data Visualization Packages

# Matplotlib and Seaborn
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
%matplotlib inline
import seaborn as sns
sns.set(rc={'image.cmap': 'cubehelix'})
sns.set_context('poster')
figure(figsize=(20, 16))

# Bokeh
import bokeh as bk
from bokeh.io import output_notebook, show, output_file, save
from bokeh.plotting import figure, show, output_file, save
from bokeh.models import (ColumnDataSource as cds, Plot, DatetimeAxis, PanTool, WheelZoomTool, HoverTool, 
                          tickers, BoxAnnotation, Panel, Range1d, LabelSet, Label, NumeralTickFormatter, 
                          LogColorMapper, GeoJSONDataSource, LinearColorMapper, ColorBar,
                          LogTicker, BasicTicker, CategoricalColorMapper, FixedTicker, AdaptiveTicker)
from bokeh.palettes import viridis, magma, inferno, cividis, Greens, Blues, PuRd, YlOrRd, YlOrBr, RdYlGn
from bokeh.embed import file_html
from bokeh.layouts import layout, gridplot
output_notebook()

In [ ]:
# Import Machine Learning Packages
from sklearn import base
from sklearn import metrics
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, ShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn import pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import normalize, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.svm import SVR
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer

## Import Datasets

In [ ]:
####################### 1.) INPUT ORIGINAL REPLICA DATASET #############################
# Read in the full summary Replica File 
replica_file = 'seeds_ii_replica.csv'
replica = pd.read_csv(replica_file,thousands=',',low_memory=False)

# Get information about the dataset
display(replica.shape)
replica.info()

In [ ]:
####################### 2.) ADD IN A GPD FILE TO FIGURE OUT LATITUDE CENTROIDS
latlong = gpd.read_file('/Users/rohithdesikan/Desktop/Data Analysis/General Data Sources/Low Income Solar/high_mf_own/high_mf_own.shp')


In [ ]:
latlong.iloc[:24253,:].to_csv('latlong1.csv')
latlong.iloc[24253:48506,:].to_csv('latlong2.csv')
latlong.iloc[48506:,:].to_csv('latlong3.csv')

In [ ]:
####################### 3.) INSERT $ SAVED PER YEAR DATA INTO DATASET #############################
# Add in solar potential bill savings (This is y aka labels)
lmi_savings = pd.read_csv('lmi_potential_electric_bill_savings.csv')

In [ ]:
####################### 4.) INSERT PV ROOFTOP POTENTIAL BY TRACT #############################
pv_gen = pd.read_csv('pv_rooftop_tract_lmi_gwh.csv')


## Clean up Data

In [ ]:
####################### 1.) CLEAN UP ORIGINAL REPLICA DATASET #############################

# Drop unrequired columns 
drop_cols1 = ['gisjoin','state_fips','state_name','county_fips','tract_fips',
              'aqi_max_description','company_na',
              'aqi_90th_percentile_description','aqi_median_description',
              'climate_zone','eia_id','avg_cbi_usd_p_w','avg_ibi_pct',
              'hdd_std','hdd_ci','cdd_std','cdd_ci','lihtc_qualified']
replica_drop = replica.drop(drop_cols1,axis='columns')

# Drop columns that are more than 75% NAN
replica_nans = replica_drop.dropna(axis='columns', thresh=int(0.75*len(replica)))

In [ ]:
# 2.) Find annual consumption and annual savings instead of monthly and rename the columns (Multiply by 12*1.1 Safety)
replica_nans['avg_monthly_consumption_kwh'] *= 12*1.1
replica_nans['avg_monthly_bill_dlrs'] *= 12*1.1

replica_nans.rename({'avg_monthly_consumption_kwh':'avg_yearly_consumption_kwh',
           'avg_monthly_bill_dlrs':'avg_yearly_bill_dlrs'},
            axis=1,inplace=True)

# Move these specific columns to the end of the matrix
cols1 = replica_nans.columns.tolist()
n = int(cols1.index('avg_yearly_consumption_kwh'))
cols1 = cols1[:n] + cols1[n+3:] + cols1[n:n+3]
replica_nans = replica_nans[cols1]

In [ ]:
####################### 3.) MATCH GEOID TO ZIP CODES WITH DATASET #2 #########################
replica_geom = replica_nans.copy()
mp = latlong['geometry']
centroid_y = latlong['geometry'].centroid.y

replica_geom['centroid_y'] = centroid_y
replica_geom['polygons'] = mp

cols1 = replica_geom.columns.tolist()
n = int(cols1.index('centroid_x'))
cols1 = cols1[:n+1]+ cols1[-2:] + cols1[n+1:-2]
replica_geom = replica_geom[cols1]

In [ ]:
####################### 3.) Sort and only choose the geoid and potential solar gen columns ##########
pv_gen =  pv_gen[['geoid','lmi_mf_gwh', 'lmi_sf_gwh', 'lmi_rent_gwh', 'lmi_own_gwh']]
replica_pv = replica_geom.merge(pv_gen,how='left',on='geoid')

In [ ]:
####################### 4.) Sort and only choose the geoid and potential annual bill savings columns ##########
lmi_savings = lmi_savings[['geoid','lmi_potential_savings_dlrs_year']]
replica_final = replica_pv.merge(lmi_savings,how='left',on='geoid')
replica_final.rename({'lmi_potential_savings_dlrs_year':'Savings [$/yr]'},axis=1,inplace=True)
replica_final['Savings [% Annual Income]'] = (replica_final['Savings [$/yr]']*100)/replica_final['hh_med_income']

########################## ########################## ########################## ########################## 

In [ ]:
################# DILL REPLICA_FINAL TO PULL IN LATER
# dill.dump(replica_final, open('replica_final.pkd', 'wb'))

replica_final = dill.load(open('replica_final.pkd', 'rb'))

In [ ]:
display(replica_final.shape)
display(replica_final)

## Create subsets of data for visualization

In [ ]:
# Chose the demographic subset of the data
replica_cols = replica_final.columns.tolist()

plys = replica_cols.index('polygons')
cty = replica_cols.index('company_ty')
locale = replica_cols.index('locale')

r_base = replica_final.iloc[:,:plys]
r_dems = replica_final.iloc[:,cty:locale+1]

rdems = pd.concat([r_base,r_dems],axis=1)
rdems[['Savings [$/yr]','Savings [% Annual Income]']] = replica_final.iloc[:,-2:]

# Work with the Demographic Subset to get Mean HH Income and Total Population
rdems2 = replica_final[['geoid','state_abbr','county_name','polygons','hh_med_income','pop_total']]

In [ ]:
# Choose the lidar subset of the data
r_lid = replica_final.iloc[:,:cty]
r_end = replica_final.iloc[:,locale+1:]
rlidar = pd.concat([r_lid,r_end],axis=1)

# Work with the Lidar Subset to get dev roof area and MWH gen potential
rlidar2 = rlidar.copy()


# Choose the Multifamily vs Single Family subset of the data
devp_m2_cols = [x for x in rlidar2.columns[rlidar2.columns.str.contains('devp_m2')]]
hh_cols = [x for x in rlidar2.columns[rlidar2.columns.str.contains('hh')]]
mwh_cols = [x for x in rlidar2.columns[rlidar2.columns.str.contains('mwh')]]

rlidar2['hh'] = rlidar2[hh_cols].sum(axis=1)
rlidar2['devp_m2'] = rlidar2[devp_m2_cols].sum(axis=1)
rlidar2['mwh'] = rlidar2[mwh_cols].sum(axis=1)
rlidar2 = rlidar2[['hh','devp_m2','mwh','Savings [% Annual Income]']]

In [ ]:
# Concatenate:
rplot = pd.concat([rdems2,rlidar2],axis=1)

# Data Visualization Inputs and Functions

In [ ]:
# Define a constant style for all plots
def style(p):
        # Title 
        p.title.align = 'center'
        p.title.text_font_size = '16pt'
        p.title.text_font = 'sans serif'

        # Axis titles
        p.xaxis.axis_label_text_font_size = '12pt'
        p.xaxis.axis_label_text_font_style = 'bold'
        p.yaxis.axis_label_text_font_size = '12pt'
        p.yaxis.axis_label_text_font_style = 'bold'

        # Tick labels
        p.xaxis.major_label_text_font_size = '10pt'
        p.yaxis.major_label_text_font_size = '10pt'

        return p

In [ ]:
# Pick a state: (#####---- CODE ONLY WORKS FOR 1 STATE -----#####)
statef = 'Texas' # Pick a state
state_abbr = list(mapdf.loc[mapdf['STATE_NAME'].isin([statef]),'STATE_ABBR']) # Find the abbreviation

# Find the map and replica as dataframes
mapstate_df = mapdf[mapdf['STATE_NAME'].isin([statef])]
rstate_df = rgeo[rgeo['state_abbr'] == state_abbr[0]]

# Convert the state_dataframe to a json object
mapjson = mapstate_df.to_json()
rjson = rstate_df.to_json()

# Find the latitude and longitude from the base map dataframe
latmin = round(float(mapstate_df['geometry'].bounds['miny']))-1
latmax = round(float(mapstate_df['geometry'].bounds['maxy']))+1
longmin = round(float(mapstate_df['geometry'].bounds['minx']))-1
longmax = round(float(mapstate_df['geometry'].bounds['maxx']))+1
longdist = abs(longmin-longmax)
latdist = latmax-latmin

if longdist>latdist:
    fig_width=1000
    fig_height = int(fig_width*(latdist/longdist))
elif latdist>longdist:
    fig_height=1000
    fig_width = int(fig_height*(longdist/latdist))

# Machine Learning Ensemble Model

In [ ]:
replica_f = replica_final.fillna(method='ffill', axis = 0)

# Find the columns within the data and send them to a list
replica_cols = replica_f.columns.tolist()

# Split at company_ty which is the split between lidar and demographic
cty = replica_cols.index('company_ty')

r_lidar = replica_f.iloc[:,:cty]
r_lidar.drop(['state_abbr', 'county_name', 'centroid_x', 'centroid_y', 'polygons'], axis = 1, inplace = True)
r_demographic = replica_f.iloc[:,cty:-2]

savings = replica_f['Savings [$/yr]']
# savings = [0 if a<0 else a for a in savings]

In [ ]:
# Set up Lidar data ML sets 
X_train_lidar, X_test_lidar, y_train_lidar, y_test_lidar = train_test_split(r_lidar, 
                                                                            savings, 
                                                                            test_size = 0.2, 
                                                                            random_state = 42)
y_train_lidar_indices = y_train_lidar.index.values.tolist()
y_test_lidar_indices = y_test_lidar.index.values.tolist()
display(X_train_lidar.shape)
display(len(y_train_lidar))
display(X_test_lidar.shape)
display(len(y_test_lidar))

In [ ]:
# Set up demographic ML sets
X_train_dem, X_test_dem,  y_train_dem, y_test_dem = train_test_split(r_demographic, 
                                                                    savings, 
                                                                    test_size = 0.2, 
                                                                    random_state = 42)


train_one_hot = pd.get_dummies(X_train_dem[['company_ty',
                                        'climate_zone_description', 
                                        'moisture_regime',
                                        'locale']])
X_train_dem = X_train_dem.join(train_one_hot)
X_train_dem.drop(['company_ty', 'climate_zone_description', 'moisture_regime', 'locale'], axis = 1, inplace = True)

test_one_hot = pd.get_dummies(X_test_dem[['company_ty',
                                        'climate_zone_description', 
                                        'moisture_regime',
                                        'locale']])
X_test_dem = X_test_dem.join(test_one_hot)
X_test_dem.drop(['company_ty', 'climate_zone_description', 'moisture_regime', 'locale'], axis = 1, inplace = True)

y_train_dem_indices = y_train_lidar.index.values.tolist()
y_test_dem_indices = y_test_lidar.index.values.tolist()

display(X_train_dem.shape)
display(len(y_train_dem))
display(X_test_dem.shape)
display(len(y_test_dem))

In [ ]:
lidar_gbr = GradientBoostingRegressor(n_estimators= 600, 
                                      max_depth= 12, 
                                      min_samples_split= 2,
                                      learning_rate= 0.02, 
                                      loss= 'lad')

lidar_gbr.fit(X_train_lidar, y_train_lidar)
ypred_train_gbr_lidar = lidar_gbr.predict(X_train_lidar)
ypred_test_gbr_lidar = lidar_gbr.predict(X_test_lidar)

In [ ]:
################# Get Root Mean Squared Error (RMSE) value as the evaluation metric
RMSE_train_gbr_lidar = metrics.mean_absolute_error(y_train_lidar, ypred_train_gbr_lidar)
RMSE_test_gbr_lidar = metrics.mean_absolute_error(y_test_lidar, ypred_test_gbr_lidar)
display((RMSE_train_gbr_lidar,RMSE_test_gbr_lidar))

In [ ]:
################# Plot train or test data to see how close it is
plt.figure(figsize=(10, 8))
plt.plot(pd.Series.rolling(pd.Series(y_train_lidar.values.tolist()), window=500).mean())
plt.plot(pd.Series.rolling(pd.Series(ypred_train_gbr_lidar), window=500).mean())

In [ ]:
################# Plot train or test data to see how close it is
plt.figure(figsize=(10, 8))
plt.plot(pd.Series.rolling(pd.Series(y_test_lidar.values.tolist()), window=500).mean())
plt.plot(pd.Series.rolling(pd.Series(ypred_test_gbr_lidar), window=500).mean())

In [ ]:
# RF Regression for Demographic Data
demographic_rf = RandomForestRegressor(n_estimators = 300, max_depth = 8)
demographic_rf.fit(X_train_dem, y_train_dem)
ypred_train_dem = demographic_rf.predict(X_train_dem)
ypred_test_dem = demographic_rf.predict(X_test_dem)

In [ ]:
################# Get Root Mean Squared Error (RMSE) value as the evaluation metric
RMSE_train_dem = metrics.mean_absolute_error(y_train_dem, ypred_train_dem)
RMSE_test_dem = metrics.mean_absolute_error(y_test_dem, ypred_test_dem)
display((RMSE_train_dem,RMSE_test_dem))

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(pd.Series.rolling(pd.Series(y_train_dem), window=500).mean())
plt.plot(pd.Series.rolling(pd.Series(ypred_train_dem), window=500).mean())

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(pd.Series.rolling(pd.Series(y_test_dem), window=500).mean())
plt.plot(pd.Series.rolling(pd.Series(ypred_test_dem), window=500).mean())

In [ ]:
# dill.dump(ypred_train_gbr_lidar, open('ypred_train_gbr_lidar.pkd', 'wb'))
# dill.dump(ypred_test_gbr_lidar, open('ypred_test_gbr_lidar.pkd', 'wb'))
# dill.dump(ypred_train_dem, open('ypred_train_dem.pkd', 'wb'))
# dill.dump(ypred_test_dem, open('ypred_test_dem.pkd', 'wb'))

ypred_train_gbr_lidar = dill.load(open('ypred_train_gbr_lidar.pkd', 'rb'))
ypred_test_gbr_lidar = dill.load(open('ypred_test_gbr_lidar.pkd', 'rb'))
ypred_train_dem = dill.load(open('ypred_train_dem.pkd', 'rb'))
ypred_test_dem = dill.load(open('ypred_test_dem.pkd', 'rb'))

In [ ]:
X_train_ensemble = np.vstack((ypred_train_gbr_lidar, ypred_train_dem)).T
y_train_ensemble = y_train_lidar.values.tolist()
X_test_ensemble = np.vstack((ypred_test_gbr_lidar, ypred_test_dem)).T
y_test_ensemble = y_test_lidar.values.tolist()

In [ ]:
# Build the ensemble model
ensemble_linear = Ridge(alpha = 2, fit_intercept = True)
ensemble_linear.fit(X_train_ensemble, y_train_ensemble)
ypred_train_ensemble = ensemble_linear.predict(X_train_ensemble)
ypred_test_ensemble = ensemble_linear.predict(X_test_ensemble)

In [ ]:
################# Get Root Mean Squared Error (RMSE) value as the evaluation metric
RMSE_train_ensemble = metrics.mean_absolute_error(y_train_ensemble, ypred_train_ensemble)
RMSE_test_ensemble = metrics.mean_absolute_error(y_test_ensemble, ypred_test_ensemble)
display((RMSE_train_ensemble, RMSE_test_ensemble))

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(pd.Series.rolling(pd.Series(y_train_ensemble), window=500).mean())
plt.plot(pd.Series.rolling(pd.Series(ypred_train_ensemble), window=500).mean())

In [ ]:
plt.figure(figsize=(10, 8))
plt.plot(pd.Series.rolling(pd.Series(y_test_ensemble), window=500).mean())
plt.plot(pd.Series.rolling(pd.Series(ypred_test_ensemble), window=500).mean())

## Set up plots

In [ ]:
solar_irradiation_cols = [x for x in replica_final.columns[replica_final.columns.str.contains('gwh')]]
replica_final['gwh'] = replica_final[solar_irradiation_cols].sum(axis=1)

In [ ]:
replica_small = replica_final[['geoid', 'state_abbr', 'county_name' , 'polygons', 'avg_yearly_bill_dlrs', 'gwh', 'Savings [$/yr]']]
replica_train = replica_small.iloc[y_train_lidar_indices, :]
replica_train['Savings_Predictions'] = ypred_train_ensemble

replica_test = replica_small.iloc[y_test_lidar_indices, :]
replica_test['Savings_Predictions'] = ypred_test_ensemble
replica_plot = pd.concat([replica_train, replica_test], axis = 0)

# Turning replica data frame into a geopandas dataframe
replica_plot.rename({'polygons':'geometry'},inplace=True,axis=1)
replica_geoplot = gpd.GeoDataFrame(replica_plot, geometry='geometry')

## Plot entire USA Map on Matplotlib and each state on Bokeh

In [ ]:
# USA Basemap File
file = '/Users/rohithdesikan/Desktop/Data Analysis/The Data Incubator/Capstone Project/states_21basic/states.shp'
map_df = gpd.read_file(file)

In [ ]:
# Matplotlib Single Variable Plot of the entire US
basem = map_df.plot(figsize=(300, 200), color='white', edgecolor='black')

replica_geoplot.plot(ax=basem,column='Savings_Predictions',cmap='YlGn',scheme='quantiles',legend=True)


# ALL OF USA
plt.xlim(-125,-65)
plt.ylim(25,50)

plt.show()

plt.savefig('USA Solar Savings Basemap.jpg')

In [ ]:
# Make the geoJSON datasource
rjson = replica_few_plot.to_json()

geo_source = GeoJSONDataSource(geojson=rjson)

# Bokeh Plot of Predictions
n=6 # Number of colors on the choropleth and legend

# Set the colormapper
Greens1 = Greens[n]
Greens2 = Greens1[::-1]
cmapper = LinearColorMapper(palette=Greens2,
                            low = replica_geoplot['Savings_Predictions'].min(),
                            high = replica_geoplot['Savings_Predictions'].max())


low = replica_geoplot['Savings_Predictions'].min()
high = replica_geoplot['Savings_Predictions'].max()
diff = high - low
ticks = [diff/6, diff/3, diff/2, 2*diff/3, 5*diff/6, diff]
    
color_bar = bk.models.ColorBar(color_mapper=cmapper, ticker=FixedTicker(minor_ticks = [], ticks = ticks),
                 label_standoff=12, border_line_color=None, location=(0,0))

# Set up the figure 
p = figure(
    title='Savings Predictions', 
    x_axis_location=None, 
    y_axis_location=None,
    x_range=(-125,-70),
    y_range=(31,43),
    tools="pan,wheel_zoom,box_zoom,reset,hover,save"
)

# Disable grid lines
p.grid.grid_line_color = None

# Plot the required variable
p.patches('xs', 
          'ys', 
          source=geo_source,
          fill_color={'field': 'Savings_Predictions', 'transform': cmapper},
          fill_alpha=0.4, 
          line_color="white", 
          line_width=1)

p.add_tools(HoverTool(
    tooltips= [
    ("State",'@state_abbr'),
    ("County Name", "@county_name"),
    ("Savings Predictions", "@Savings_Predictions{int}"),
    ("(Long, Lat)", "($x, $y)")
    ],
    formatters={
    "@State":"printf",
    "@county_name": "printf",
    "@Savings_Predictions": "numeral",
    "($x, $y)": "numeral"
    },
    mode='mouse'
))

p.add_layout(color_bar, 'right')
# p.legend.location = "top_right"
# p.legend.click_policy="hide"

style(p)
show(p)